<a href="https://colab.research.google.com/github/963SY/library_children_sample_en/blob/main/27_08_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hausarbeit – Lineare Regression zur Vorhersage von **borrowed_books**

Dieses Notebook ist **prüfungsfertig** und umfasst:
- **Datenvorverarbeitung** (Missing Values, Ausreißer/Winsorize, One‑Hot‑Encoding)
- **Partitionierung** (Train/Test) & **Skalierung**
- **Lineares Regressionsmodell**
- **Annahmenprüfung** (Residuen, QQ-Plot, VIF)
- **Evaluation** (R² & RMSE, In- & Out-of-Sample)
- **Visualisierungen** mit Matplotlib (einfaches, neutrales Styling)



> **Hinweis (Pakete):**
```bash
conda install pandas matplotlib scikit-learn statsmodels
# oder
pip install pandas matplotlib scikit-learn statsmodels
```


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression # Corrected import
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings, re
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


## 1) Daten laden & Spaltennamen bereinigen
- Lädt `library_children.csv` (legen Sie die Datei ins gleiche Verzeichnis).
- Spaltennamen werden **getrimmt**, in **Kleinschreibung** konvertiert und **Leerzeichen → Unterstrich**.


In [5]:
#uploading path
path = "library_children.csv"
df = pd.read_csv(path, sep=';')

# Spalten bereinigen
clean_cols = [re.sub(r"\s+", "_", str(c)).strip("_").lower() for c in df.columns]
df.columns = clean_cols

print("Spalten nach Bereinigung:", list(df.columns))
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'library_children.csv'

In [ ]:
# Zielspalte automatisch finden (enthält 'borrow')
target_col = None
for c in df.columns:
    if 'borrow' in c:
        target_col = c
        break
if target_col is None:
    raise KeyError("Keine Zielspalte gefunden (es wird eine Spalte benötigt, deren Name 'borrow' enthält).")

# Spalten, die typischerweise nicht als Features verwendet werden
drop_if_exists = ['child_id', 'book_return_status']

# Sauber löschen, falls vorhanden
for c in drop_if_exists:
    if c in df.columns:
        df.drop(columns=[c], inplace=True)

print("Zielspalte (Target):", target_col)

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols_wo_traget= [c for c in num_cols if c != target_col]
cat_cols = [c for c in df.columns if c not in num_cols]
for c in num_cols_wo_traget:
    if df[c].isna().any():
        df[c].fillna(df[c].median(), inplace=True)


for c in cat_cols:
    if df[c].isna().any():
        df[c].fillna(df [c].mode(), inplace=True)

df=df.dropna(subset=[target_col])
print("Zeilen nach Missing-Handling:", df.shape[0])


In [ ]:
#check for duplicate rows
print("Number of duplicate rows before removal:", df.duplicated().sum())

#Display some duplicate rows (if any)
print("Sample duplicate rows:")
display(df[df.duplicated()].head())

#Remove duplicate rows (keep the first occurrence)
df.drop_duplicates(inplace=True)

#Verify removal of duplicates
print("Number of rows after removing duplicates:", df.shape[0])
print("Number of duplicate rows now:", df.duplicated().sum())

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# Convert 'fine_amount' to numeric, handling commas
df['fine_amount'] = df['fine_amount'].astype(str).str.replace(',', '.', regex=False)
df['fine_amount'] = pd.to_numeric(df['fine_amount'], errors='coerce')

# Apply one-hot encoding to 'gender' column
if 'gender' in df.columns:
    df_encoded = pd.get_dummies(df, columns=['gender'], drop_first=True)
else:
    df_encoded = df.copy()

# Calculate and display correlation
display(df_encoded.corr(numeric_only=True))

In [ ]:
# Convert 'fine_amount' to numeric, handling commas
df['fine_amount'] = df['fine_amount'].astype(str).str.replace(',', '.', regex=False)
df['fine_amount'] = pd.to_numeric(df['fine_amount'], errors='coerce')

# Apply one-hot encoding to 'gender' column
if 'gender' in df.columns:
    df_encoded = pd.get_dummies(df, columns=['gender'], drop_first=True)
else:
    df_encoded = df.copy()

# Generate the heatmap using the encoded DataFrame
sns.heatmap(df_encoded.corr(), annot=True, cmap="plasma")
plt.show()

In [ ]:
outliers= ['borrowed_books']
plt.rcParams ['figure.figsize']
sns.boxplot(x=df['borrowed_books'], color="skyblue") # Use the 'borrowed_books' column directly
plt.title("Boxplot of Target(borrowed_books)")
plt.xlabel("borrowed_books")

plt.show()

In [ ]:
outliers= ['monthly_visits']
plt.rcParams ['figure.figsize'] = [8,6]
sns.boxplot(x=df['monthly_visits'], color="skyblue") # Changed to string
plt.title("Boxplot of monthly_visits") # Updated title
plt.xlabel("monthly_visits") # Updated x-label

plt.show()

In [ ]:
outliers= ['fine_amount']
plt.rcParams ['figure.figsize'] = [8,6]
sns.boxplot(x=df['fine_amount'], color="red")
plt.title("Boxplot of fine_amount")
plt.xlabel("fine_amount")

plt.show()

In [ ]:
Q1 = np.percentile (df['borrowed_books'], 25)
Q3 = np.percentile (df['borrowed_books'], 75) # Added comma
IQR = Q3 - Q1
Lower_Bound = Q1 - 1.5 * IQR
Upper_Bound = Q3 + 1.5 * IQR
print("Lower_Bound", Lower_Bound)
print("Upper_Bound", Upper_Bound)

In [ ]:
df_no_outliers = df[(df['borrowed_books'] >= Lower_Bound) & (df['borrowed_books'] <= Upper_Bound)]
df_no_outliers

In [ ]:
print(" Zeilen vor Ausreißern:", df.shape[0])
print (" Zeilen nach Ausreißern:", df_no_outliers.shape[0])

In [ ]:
sns.boxplot(x=df_no_outliers['borrowed_books'])
plt.title("Boxplot nach Ausreißern")
plt.show()

In [ ]:
Q1 = np.percentile (df['fine_amount'], 25)
Q3 = np.percentile (df['fine_amount'], 75)
IQR = Q3 - Q1
Lower_Bound = Q1 - 1.5 * IQR
Upper_Bound = Q3 + 1.5 * IQR
print("Lower_Bound", Lower_Bound)
print("Upper_Bound", Upper_Bound)

In [ ]:
df_no_outliers = df[(df['fine_amount'] >= Lower_Bound) & (df['fine_amount'] <= Upper_Bound)]

In [ ]:
print ("zeilen vor Ausreißern:", df.shape[0])
print (" Zeilen nach Ausreißern:", df_no_outliers.shape[0])

In [ ]:
sns.boxplot(x=df_no_outliers['fine_amount'])
plt.title("Boxplot nach Ausreißern")
plt.show()


## 5) Dummifizierung (One‑Hot‑Encoding) für `gender`
- Verhindert perfekte Multikollinearität durch `drop_first=True`.


In [ ]:

if 'gender' in df.columns:
    df = pd.get_dummies(df, columns=['gender'], drop_first=True)
display(df.head())



## 6) Feature‑Matrix & Zielvariable, Train/Test‑Split, Skalierung


In [ ]:
X = df.drop(columns=[target_col])
y = df[target_col]

# Convert 'fine_amount' to numeric here before splitting and scaling
if 'fine_amount' in X.columns:
    X['fine_amount'] = X['fine_amount'].astype(str).str.replace(',', '.', regex=False)
    X['fine_amount'] = pd.to_numeric(X['fine_amount'], errors='coerce')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_cols_X = X_train.select_dtypes(include=[np.number]).columns.tolist()
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
# Only scale numeric columns if there are any
if num_cols_X:
    X_train_scaled[num_cols_X] = scaler.fit_transform(X_train[num_cols_X])
    X_test_scaled[num_cols_X] = scaler.transform(X_test[num_cols_X])


print("Train/Test Größen:", X_train.shape, X_test.shape)


## 7) Lineares Regressionsmodell & Gütemaße
- In‑Sample (Train) & Out‑of‑Sample (Test): **R²** und **RMSE**.


In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_train_pred = lr.predict(X_train_scaled)
y_test_pred  = lr.predict(X_test_scaled)

r2_train  = r2_score(y_train, y_train_pred)
r2_test   = r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test  = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train R²: {r2_train:.3f} | Test R²: {r2_test:.3f}")
print(f"Train RMSE: {rmse_train:.3f} | Test RMSE: {rmse_test:.3f}")


## 8) Annahmen prüfen
- **Residuenplot** (Homogenität, Musterfreiheit)  
- **QQ‑Plot** der Residuen (Normalitätsannahme)  
- **VIF** (Multikollinearität)


In [ ]:
# Residuen (Test)
residuals = y_test - y_test_pred

# Residuen vs. Vorhersage
plt.figure(figsize=(8,5))
plt.scatter(y_test_pred, residuals, alpha=0.7)
plt.axhline(0, linestyle='--')
plt.xlabel("Vorhergesagte Werte")
plt.ylabel("Residuen")
plt.title("Residuenplot (Testdaten)")
plt.tight_layout()
plt.show()

# QQ-Plot
fig = sm.qqplot(residuals, line='s')
plt.title("QQ-Plot der Residuen (Test)")
plt.show()

# VIF (auf Trainingsdaten, mit Konstante)
X_vif = X_train_scaled.copy()
# Convert boolean columns to numeric (int) before calculating VIF
for col in X_vif.columns:
    if X_vif[col].dtype == 'bool':
        X_vif[col] = X_vif[col].astype(int)

X_vif_const = sm.add_constant(X_vif.values)
vif_vals = [variance_inflation_factor(X_vif_const, i+1) for i in range(len(X_vif.columns))]
vif_df = pd.DataFrame({"Feature": X_vif.columns, "VIF": vif_vals}).sort_values("VIF", ascending=False)
display(vif_df.head(15))


## 9) Evaluation – Visualisierungen
- **Tatsächlich vs. Vorhergesagt** (Test)  
- **Fehlerverteilung (Histogramm)**


In [ ]:

# Tatsächlich vs. Vorhergesagt
plt.figure(figsize=(8,5))
plt.scatter(y_test, y_test_pred, alpha=0.7)
min_v, max_v = min(y_test.min(), y_test_pred.min()), max(y_test.max(), y_test_pred.max())
plt.plot([min_v, max_v], [min_v, max_v], linestyle='--')
plt.xlabel("Tatsächliche Werte")
plt.ylabel("Vorhergesagte Werte")
plt.title("Tatsächlich vs. Vorhergesagt (Test)")
plt.tight_layout()
plt.show()

# Histogramm der Residuen
plt.figure(figsize=(8,5))
plt.hist(residuals, bins=20)
plt.title("Verteilung der Residuen (Test)")
plt.xlabel("Residuum")
plt.ylabel("Häufigkeit")
plt.tight_layout()
plt.show()



## 10) Fazit
Das Notebook erfüllt die geforderten Bausteine der Hausarbeit:
**Vorverarbeitung**, **Modellierung**, **Annahmenprüfung** und **Evaluation**.  
Die Metriken (R², RMSE) geben Auskunft über die Güte der **In‑Sample** und **Out‑of‑Sample**‑Vorhersagen.  
Je nach Datenlage sind Erweiterungen sinnvoll (z. B. Regularisierung, Feature‑Engineering oder weitere erklärende Variablen).


In [ ]:
if 'gender' in df.columns:
    df = pd.get_dummies(df, columns=['gender'], drop_first=True)
display(df.head())

In [ ]:
#uploading path
path = "library_children.csv"
df = pd.read_csv(path, sep=';')

# Spalten bereinigen
clean_cols = [re.sub(r"\s+", "_", str(c)).strip("_").lower() for c in df.columns]
df.columns = clean_cols

print("Spalten nach Bereinigung:", list(df.columns))
display(df.head())

In [ ]:
# Zielspalte automatisch finden (enthält 'borrow')
target_col = None
for c in df.columns:
    if 'borrow' in c:
        target_col = c
        break
if target_col is None:
    raise KeyError("Keine Zielspalte gefunden (es wird eine Spalte benötigt, deren Name 'borrow' enthält).")

# Spalten, die typischerweise nicht als Features verwendet werden
drop_if_exists = ['child_id', 'book_return_status']

# Sauber löschen, falls vorhanden
for c in drop_if_exists:
    if c in df.columns:
        df.drop(columns=[c], inplace=True)

print("Zielspalte (Target):", target_col)

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

# Zielspalte nicht anfassen, falls numerisch
num_cols_wo_target = [c for c in num_cols if c != target_col]

for c in num_cols_wo_target:
    if df[c].isna().any():
        df[c].fillna(df[c].median(), inplace=True)

for c in cat_cols:
    if df[c].isna().any():
        df[c].fillna('unbekannt', inplace=True)

# Ziel: falls NA, löschen (optional auch Median möglich, aber hier entfernen wir wenige Fälle)
df = df.dropna(subset=[target_col])

print("Zeilen nach Missing-Handling:", df.shape[0])

In [ ]:
if 'gender' in df.columns:
    df = pd.get_dummies(df, columns=['gender'], drop_first=True)
display(df.head())